In [1]:
#!pip install quandl
H={'/home/user/HADOOP': 'C:\HADOOP',
  '/home/user/HADOOP/spark-2.3.0-bin-hadoop2.7': 'C:\HADOOP\spark-2.3.0-bin-hadoop2.7',
  '/home/user/Program Files/java/8.0.172-zulu': 'C:\Program Files\Java\jdk1.8.0_172',
  '/home/user/TMP/SPARK': 'C:\TMP\SPARK'}
H=dict([(k,k) for k in H.keys()])
Z="ok"
SB="SANDBOX.USER"
DB="MY_DATA"
TB="CURRENCY_KAFKA"
BA="SANDBOX.USER"
AB="REQUEST_CURRENCY"
CB="SANDBOX.USER"
BC="RESPONSE_CURRENCY"
DIR="/home/user/ALBINA/SASHA/SOAP"
C=[("errcode","BIGINT","cast(errcode as bigint)")   
  ,("prefix","","")
  ,("itemnum","BIGINT","cast(itemnum as bigint)")
  ,("item","","")
  ,("suffix","","")
  ,("remaining","BIGINT","cast(remaining as bigint)")
  ,("nextkey","","")
]
D=[("errcode","BIGINT","cast(errcode as bigint)")
  ,("prefix","","")
  ,("itemnum","BIGINT","cast(itemnum as bigint)")
  ,("item","","")
  ,("suffix","","")
  ,("remaining","BIGINT","cast(remaining as bigint)")
  ,("nextkey","","")
]
AKEY="mqZagQhmMKp1w1-565-E"

In [ ]:
#%%time
def parse_html(fname, nrows=-1):
    import pandas as pd
    fn = fname.replace(".html", ".csv")
    return pd.read_csv(fn)

def parse_html0(fname, nrows=-1):
    import pandas as pd
    # /html/body/section[1]/div/div/div/div[1454]/div[1]: 928
    # /html/body/section[1]/div/div/div/div[1454]/div[2]: 0350000-0449999
    # /html/body/section[1]/div/div/div/div[1454]/div[3]: Краснодар  Краснодарский край
    # /html/body/section[1]/div/div/div/div[1454]/div[4]: ОАО "Мегафон" Кавказский филиал
    # /html/body/section[1]/div/div/div/div[1454]/div[4]/p: 29.03.2010-01.01.2038
    d = pd.DataFrame(columns=['prefix', 'fromto', 'area', 'vendor', 'dates'])
    import lxml.html as LH
    import lxml.etree as LE
    import codecs
    import re
    a=[]
    m={}
    with codecs.open(DIR+"/"+"/defcodes.html", "r", 'utf-8') as rf:
        text = rf.read()
        root = LH.fromstring(text)
        tree = LE.ElementTree(root)
        s = set([])
        for e in root.iter():
            t = tree.getpath(e)
            if t.startswith("/html/body/section[1]/div/div/div/div["):
                s.add(t)
        a = list(sorted(s))
        m = dict([(re.sub("/div.*$", ""
                , x.replace("/html/body/section[1]/div/div/div/div[", "", 1).replace("]", "", 1)), 1) for x in a])

        for k in m.keys():
            row = []
            if len(d) >= nrows and nrows >= 0: break
            s = ""
            for v in tree.xpath("/html/body/section[1]/div/div/div/div["+k+"]/div[1]" + '/text()'):
                s += v.lstrip().rstrip()
            row.append(s)
            s = ""
            for v in tree.xpath("/html/body/section[1]/div/div/div/div["+k+"]/div[2]" + '/text()'):
                s += v.lstrip().rstrip()
            row.append(s)
            s = ""
            for v in tree.xpath("/html/body/section[1]/div/div/div/div["+k+"]/div[3]" + '/text()'):
                s += v.lstrip().rstrip()
            row.append(s)
            s = ""
            for v in tree.xpath("/html/body/section[1]/div/div/div/div["+k+"]/div[4]" + '/text()'):
                s += v.lstrip().rstrip()
            row.append(s)
            s = ""
            for v in tree.xpath("/html/body/section[1]/div/div/div/div["+k+"]/div[4]/p" + '/text()'):
                s += v.lstrip().rstrip()
            row.append(s)
            d.loc[len(d)+1] = row
    return d
print(Z)

In [2]:
import pandas as pd
view = parse_html0(DIR+"/"+"/defcodes.html", nrows=100)
display(view[view.index < 5])

print(Z)

ok
Wall time: 0 ns


In [ ]:
%%time

import os
os.environ['HADOOP_HOME'] = 'C:\HADOOP'
os.environ['SPARK_HOME'] = 'C:\HADOOP\spark-2.3.0-bin-hadoop2.7'
os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jdk1.8.0_172'

def start():
    import time
    from pyspark.sql import SparkSession
    res = SparkSession.builder.master('local[*]').appName('test') \
       .config('spark.sql.warehouse.dir', 'file:///' + WARE) \
       .config("spark.driver.maxResultSize", "600g") \
       .config("spark.sql.shuffle.output.partitions", "32768") \
       .getOrCreate()
    time.sleep(1)
    return res

def stop(sc):
    print("ok")
    sc.stop()

def preview(df):
    df.show(n=3, truncate=False)

WARE="C:\TMP\SPARK"
os.makedirs(WARE, exist_ok=True)
os.chmod(WARE, 0o777)
spark=start()
print(spark.version)

print(Z)

In [ ]:
%%time

def spark_table(name, sqlc):
    import pyspark.storagelevel as L
    # README https://stackoverflow.com/a/22742982
    # README https://stackoverflow.com/a/40866286
    NO_OF_EXECUTOR_INSTANCES = int(dict(sqlc.sparkContext._conf.getAll()).get("spark.executor.instances",5))
    NO_OF_EXECUTOR_CORES = int(dict(sqlc.sparkContext._conf.getAll()).get("spark.executor.cores",2))
    REPARTITION_FACTOR = 10
    ideal = NO_OF_EXECUTOR_INSTANCES * NO_OF_EXECUTOR_CORES * REPARTITION_FACTOR

    import shutil
    import pandas as pd
    if name == SB+'_'+TB:
        shutil.copyfile(DIR+"/"+"manyphones.tsv", WARE+"/ig363_"+name+".csv")
    else:
        shutil.rmtree(WARE+"/ig363_"+name+".csv")
    q = ["cast(_c"+str(i)+" as string) as " + C[i][0] for i in range(len(C))]
    qq = [(C[i][2] if len(C[i][2]) else C[i][0])+' as '+C[i][0] for i in range(len(C))]
    return sqlc.read.option("escape", "\"").option("quote", "\"").option("sep", "\t").csv(WARE+"/ig363_"+name+".csv",
                header=False).persist(L.StorageLevel.MEMORY_AND_DISK).repartition(1).selectExpr(q).selectExpr(qq)

def spark_table2(name, sqlc):
    import pyspark.storagelevel as L

    import shutil
    import pandas as pd
    if name == SB+'_'+BC:
        d = parse_html0(DIR+"/"+"defcodes.html")
        d.to_csv(DIR+"/"+"defcodes.csv", header=None, index=None)
        shutil.copyfile(DIR+"/"+"defcodes.csv", WARE+"/ig363_"+name+".csv")
    else:
        shutil.rmtree(WARE+"/ig363_"+name+".csv")
    q = ["cast(_c"+str(i)+" as string) as " + D[i][0] for i in range(len(D))]
    return sqlc.read.option("escape", "\"").option("quote", "\"").option("sep", ",").csv(WARE+"/ig363_"+name+".csv",
                header=False).persist(L.StorageLevel.MEMORY_AND_DISK).repartition(1).selectExpr(q)

def write_insert(df, dst):
    df.write.option("escape", "\"").option("quote", "\"").csv(WARE+"/ig363_"+dst, header=False, mode="Overwrite")

print(Z)

In [ ]:
#%%time

import pyspark.sql.functions as F
import pyspark.sql.window as W
import pyspark.sql.types as T

def nfunc(x):
    try:
        yyyy = '{0:04d}'.format(int(x[0:4])) if len(x) >= 4 else "1970"
        mm = '{0:02d}'.format(int(x[5:7])) if len(x) >= 7 else "01"
        dd = '{0:02d}'.format(int(x[8:10])) if len(x) >= 10 else "01"
        return yyyy + "-" + mm + "-" + dd
    except ValueError:
        return '1970-01-01' 

def get_norm_udf():
    import pyspark.sql.functions as F
    import pyspark.sql.types as T
    return F.udf(lambda x: nfunc(x), T.StringType())

df = spark_table(SB+"_"+TB, spark)
df = df.fillna({'lastdt':'1970-01-01'})
df = df.withColumn("normdt", get_norm_udf()(F.col("lastdt"))).drop("lastdt")
df = df.withColumn("lastdt", F.col("normdt")).drop("normdt")
df = df.selectExpr("phone", "lastdt", "lower(source) as source")
write_insert(df.coalesce(1), SB+"."+TB)
preview(df.filter(F.col("source") == "velobike"))

dd = spark_table2(SB+"_"+BC, spark)
write_insert(dd.coalesce(1), SB+"."+BC)
preview(dd)

dd = dd.withColumn("a", F.concat(F.lit("7"), F.col("prefix"), F.split(F.col("fromto"), "-")[0]))
dd = dd.withColumn("b", F.concat(F.lit("7"), F.col("prefix"), F.split(F.col("fromto"), "-")[1]))
dd = dd.withColumn("hash", F.concat(F.lit("7"), F.col("prefix")))
dd = dd.selectExpr("cast(a as bigint) as a", "cast(b as bigint) as b", "cast(hash as bigint) as hash", "area")
preview(dd)

dr = df.selectExpr("cast(phone as bigint) as phone", "lastdt", "source")
dr = dr.withColumn("is_2018", F.when(F.col("lastdt") >= F.lit("2018-01-01"), F.lit(1)).otherwise(F.lit(0)))
dr = dr.drop("lastdt")
dr = dr.groupBy(F.col("phone")).agg(F.max(F.col("is_2018")).alias("is_2018"),F.count("source").alias("cnt"))
dr = dr.withColumn("key", F.substring(F.col("phone"), 0, 4))
dr = dr.selectExpr("cast(key as bigint) as key", "phone", "is_2018", "cnt")
dr = dr.selectExpr("phone", "is_2018", "cnt", "key")
write_insert(dr.coalesce(1), SB+"."+CD)
preview(dr)

#res = dr.selectExpr("cast(key as bigint) as key", "phone", "is_2018", "cnt")
#res = res.join(dd, res["key"] == dd["hash"], how="left_outer")
#res = res.filter(F.col("phone") >= F.col("a"))
#res = res.filter(F.col("phone") <= F.col("b"))
#res = res.drop("a").drop("b").drop("key").drop("hash")
#res = res.withColumn("is_msc", F.when(F.col("area") == F.lit("Москва Москва"), F.lit(1)).otherwise(F.lit(0)))
#res = res.withColumn("is_mo", F.when(F.col("area") == F.lit("Москва Москва"), F.lit(1)).otherwise(F.lit(0)))
#res = res.selectExpr("phone", "is_2018", "cnt", "area", "is_msc", "is_mo")
#write_insert(res.coalesce(1), SB+"."+CD)
#preview(res)



print(Z)

In [8]:
%%time

# https://docs.quandl.com/
# https://blog.quandl.com/api-for-currency-data

import requests
answer1 = requests.get('https://www.quandl.com/api/v3/datasets/BOE/XUDLBK69/data.json?api_key='+AKEY, stream=True)
res1 = answer1.text

import pprint
pprint.pprint(res1)

answer2 = requests.get('https://www.quandl.com/api/v3/datasets/ECD/EURRUB/data.json?api_key='+AKEY, stream=True)
res2 = answer2.text

pprint.pprint(res2)

print(Z)

'{"dataset_data":{"limit":null,"transform":null,"column_index":null,"column_names":["Date","Value"],"start_date":"2005-04-01","end_date":"2018-07-09","frequency":"daily","data":[["2018-07-09",62.71],["2018-07-06",62.975],["2018-07-05",63.1075],["2018-07-04",63.2663],["2018-07-03",63.3365],["2018-07-02",63.3365],["2018-06-29",62.71],["2018-06-28",63.07],["2018-06-27",62.91],["2018-06-26",63.1996],["2018-06-25",62.829],["2018-06-22",62.968],["2018-06-21",63.74],["2018-06-20",63.621],["2018-06-19",63.8593],["2018-06-18",63.6015],["2018-06-15",63.0691],["2018-06-14",62.1823],["2018-06-13",62.424],["2018-06-12",62.8417],["2018-06-11",62.636],["2018-06-08",62.4613],["2018-06-07",62.1775],["2018-06-06",61.9828],["2018-06-05",62.29],["2018-06-04",62.0275],["2018-06-01",62.168],["2018-05-31",62.305],["2018-05-30",62.259],["2018-05-29",62.5135],["2018-05-25",62.1559],["2018-05-24",61.535],["2018-05-23",61.6],["2018-05-22",61.15],["2018-05-21",61.73],["2018-05-18",62.4175],["2018-05-17",61.8325],

('{"quandl_error":{"code":"QECx02","message":"You have submitted an incorrect '
 'Quandl code. Please check your Quandl codes and try again."}}')
ok
Wall time: 1.6 s


In [ ]:
%%time
a=[]
a.append(("load_tags", "", ""))
a.append(("load_rank", "BIGINT", "cast(load_rank as bigint)"))
b=[]
b.append(("a", "BIGINT", "cast(concat('7', prefix, split(fromto, '-')[0]) as bigint)"))
b.append(("b", "BIGINT", "cast(concat('7', prefix, split(fromto, '-')[1]) as bigint)"))
b.append(("hash", "BIGINT", "cast(concat('7', prefix) as bigint)"))
b.append(("load_tags", "", "concat('')"))
b.append(("load_rank", "BIGINT", "row_number() over()"))
c=[]
c.append(("load_tags", "", ""))
c.append(("load_rank", "BIGINT", "cast(load_rank as bigint)"))
print("(1) rename "+WARE+"/ig363_"+SB+"."+TB)
print("part.csv to")
print("many.csv")
print("")
print("")
print("(2) upload to ambari (permissions 444)")
print("make folder")
print("/user/yurbasov/tmp/phones_many")
print("permissions 777")
print("")
print("")
print("(3) CREATE EXTERNAL TABLE "+SB+"_"+TB+"(")
print('\n,'.join(['`'+C[i][0]+'`  string' for i in range(len(C))]))
print(")")
print("ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'")
print("WITH SERDEPROPERTIES (")
print("\"separatorChar\" = \",\",")
print("\"quoteChar\"     = \"\\\"\"")
print(") STORED as textfile LOCATION '/user/yurbasov/tmp/phones_many/'")
print("")
print("")
print("(4) CREATE TABLE "+SB+"_"+AB+"(")
print('\n,'.join(['`'+C[i][0]+'`  ' + (C[i][1] if len(C[i][1]) else 'string') for i in range(len(C))]))
print(",")
print('\n,'.join(['`'+a[i][0]+'`  ' + (a[i][1] if len(a[i][1]) else 'string') for i in range(len(a))]))
print(") STORED AS ORC tblproperties (\"orc.compress\"=\"SNAPPY\")")
print("")
print("")
print("(5) INSERT INTO "+SB+"_"+AB)
print("SELECT")
print('\n,'.join(['case when `'+C[i][0]+'`=\'\' then null else '+(C[i][2] if len(C[i][2]) else C[i][0])+' end as '+C[i][0] for i in range(len(C))]))
print(',"" as load_tags')
print(',row_number() over() as load_rank')
print(" FROM "+SB+"_"+TB)
print("")
print("")
print("(6) rename "+WARE+"/ig363_"+SB+"."+BC)
print("part.csv to")
print("dict.csv")
print("")
print("")
print("(7) upload to ambari (permissions 444)")
print("make folder")
print("/user/yurbasov/tmp/dicts_many")
print("permissions 777")
print("")
print("")
print("(8) CREATE EXTERNAL TABLE "+SB+"_"+BC+"(")
print('\n,'.join(['`'+D[i][0]+'`  string' for i in range(len(D))]))
print(")")
print("ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'")
print("WITH SERDEPROPERTIES (")
print("\"separatorChar\" = \",\",")
print("\"quoteChar\"     = \"\\\"\"")
print(") STORED as textfile LOCATION '/user/yurbasov/tmp/dicts_many/'")
print("")
print("")
print("(9) CREATE TABLE "+CB+"."+BC+"(")
print('\n,'.join(['`'+D[i][0]+'`  ' + (D[i][1] if len(D[i][1]) else 'string') for i in range(len(D))]))
print(",")
print('\n,'.join(['`'+b[i][0]+'`  ' + (b[i][1] if len(b[i][1]) else 'string') for i in range(len(b))]))
print(") STORED AS ORC tblproperties (\"orc.compress\"=\"SNAPPY\")")
print("")
print("")
print("(10) INSERT INTO "+CB+"."+BC)
print("SELECT")
print('\n,'.join(['case when `'+D[i][0]+'`=\'\' then null else '+(D[i][2] if len(D[i][2]) else D[i][0])+' end as '+D[i][0] for i in range(len(D))]))
print(",")
print('\n,'.join([''+(b[i][2] if len(b[i][2]) else b[i][0])+' as '+b[i][0] for i in range(len(b))]))
print(" FROM "+SB+"_"+BC)
print("")
print("")
print("(11) rename "+WARE+"/ig363_"+SB+"."+CD)
print("part.csv to")
print("result.csv")
print("")
print("")
print("(12) upload to ambari (permissions 444)")
print("make folder")
print("/user/yurbasov/tmp/results_many")
print("permissions 777")
print("")
print("")
print("(13) CREATE EXTERNAL TABLE "+SB+"_"+CD+"(")
print('\n,'.join(['`'+E[i][0]+'`  string' for i in range(len(E))]))
print(")")
print("ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'")
print("WITH SERDEPROPERTIES (")
print("\"separatorChar\" = \",\",")
print("\"quoteChar\"     = \"\\\"\"")
print(") STORED as textfile LOCATION '/user/yurbasov/tmp/results_many/'")
print("")
print("")
print("(14) CREATE TABLE "+SB+"_"+CD+"_SNAPPY(")
print('\n,'.join(['`'+E[i][0]+'`  ' + (E[i][1] if len(E[i][1]) else 'string') for i in range(len(E))]))
print(",")
print('\n,'.join(['`'+c[i][0]+'`  ' + (c[i][1] if len(c[i][1]) else 'string') for i in range(len(c))]))
print(") STORED AS ORC tblproperties (\"orc.compress\"=\"SNAPPY\")")
print("")
print("")
print("(15) INSERT INTO "+SB+"_"+CD+"_SNAPPY")
print("SELECT")
print('\n,'.join(['case when `'+E[i][0]+'`=\'\' then null else '+(E[i][2] if len(E[i][2]) else E[i][0])+' end as '+E[i][0] for i in range(len(E))]))
print(',"" as load_tags')
print(',row_number() over() as load_rank')
print(" FROM "+SB+"_"+CD)
print("")
print("")
print("(16*) CREATE TABLE "+SB+"_"+CD+"_RESULT")
print("AS")
print("WITH pre AS")
print("(SELECT a, b, area,")
print('\n,'.join(['`'+E[i][0]+'`' for i in range(len(E))]))
print(",cast(phone as bigint) as tel")
print(" FROM "+SB+"_"+CD+"_SNAPPY s")
print("LEFT JOIN")
print(CB+"."+BC+" d")
print("ON d.hash = s.key")
print(")")
print(" SELECT tel, is_2018, cnt, area, case when area = 'Москва Москва' then 1 else 0 end as is_msc FROM pre")
print("WHERE tel >= a")
print("AND  tel <= b")
print("")
print("")
print("(17*) SELECT * FROM "+SB+"_"+CD+"_RESULT"+" WHERE load_tags = ''")
print("")
print("")
print("(18*) in ambari download to hdfs")
print("/user/yurbasov/tmp/yurbasov_many_results"+".csv.gz")
print("")
print("")
print("(19*) from ambari download from hdfs")
print(DIR+"/yurbasov_many_results"+".csv.gz")
print("unpack to "+DB+"."+CD+" (.csv)")
print("")
print("")
print(Z)